In [ ]:
import pandas as pd
import mysql.connector
from sqlalchemy import types, create_engine

# Set unlimited table view
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# MySQL Connection
MYSQL_USER = 'root'
MYSQL_PASSWORD = open('/run/secrets/db-password').read().strip()
MYSQL_HOST_IP = 'db'
MYSQL_PORT = 3306
MYSQL_DATABASE = 'example'

engine = create_engine(f'mysql+mysqlconnector://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST_IP}:{MYSQL_PORT}/{MYSQL_DATABASE}', echo=False)

df = pd.read_sql_table('museums', engine.connect())
df = df.drop(labels=['index'], axis=1)

In [ ]:
df

In [ ]:
import plotly.express as px

fig = px.scatter(df, y='Visitors per year', x='Wiki languages count', hover_data=['Museum', 'City'], 
                 color='Continent',
                 trendline="ols")
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig.show()

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")

sns.pairplot(df , hue="Art/culture museum");

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

df[['City population']].values.reshape(-1, 1) 

X = df[['City population']].values.reshape(-1, 1) 
X_train, X_test, y_train, y_test = train_test_split(X, df['Visitors per year'], random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

x_range = np.linspace(X.min(), X.max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))

fig = px.scatter(df, y='Visitors per year', x='City population')

fig.add_traces(go.Scatter(x=X_train.squeeze(), y=y_train, name='train', mode='markers'))
fig.add_traces(go.Scatter(x=X_test.squeeze(), y=y_test, name='test', mode='markers'))
fig.add_traces(go.Scatter(x=x_range, y=y_range, name='prediction'))

fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))


fig.show()